# CSS 120: Environmental Data Science

## Paleoclimate

### Umberto Mignozzetti (UCSD)

(Based on Project Pythia and ClimateMatch)

# Packages


In [ ]:
# To install
# !pip install LiPD --quiet
# !pip install pyleoclim --quiet
# !pip install climlab --quiet

# Packages


In [ ]:
# System helpers
import os
import sys
from io import StringIO
import tempfile

# Data analysis
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pooch

# Principal Component Analysis
from sklearn.decomposition import PCA

# Packages


In [ ]:
# Principal Component Analysis
from sklearn.decomposition import PCA

# Paleodata analysis
import lipd
import pyleoclim as pyleo

# Climlab
from climlab import constants as const
from climlab.solar.orbital import OrbitalTable
from climlab.solar.insolation import daily_insolation

# Maps
import cartopy, cartopy.crs as ccrs, cartopy.feature as cfeature, cartopy.io.shapereader as shapereader

##  Helper functions


In [ ]:
# Pooch Load
def pooch_load(filelocation=None, filename=None, processor=None):
    shared_location = "~/"
    user_temp_cache = tempfile.gettempdir()

    if os.path.exists(os.path.join(shared_location, filename)):
        file = os.path.join(shared_location, filename)
    else:
        file = pooch.retrieve(
            filelocation,
            known_hash=None,
            fname=os.path.join(user_temp_cache, filename),
            processor=processor,
        )

    return file

##  Helper functions


In [ ]:
# Function to convert the PAGES2K LiDP files in a pandas.DataFrame
def lipd2df(
    lipd_dirpath,
    pkl_filepath=None,
    col_str=[
        "paleoData_pages2kID", "dataSetName", "archiveType", "geo_meanElev", "geo_meanLat",
        "geo_meanLon", "year", "yearUnits", "paleoData_variableName", "paleoData_units",
        "paleoData_values", "paleoData_proxy",
    ],
):
    """
    Convert a bunch of PAGES2k LiPD files to a `pandas.DataFrame` to boost data loading.

    If `pkl_filepath` isn't `None`, save the DataFrame as a pikle file.

    Parameters:
    ----------
        lipd_dirpath: str
          Path of the PAGES2k LiPD files
        pkl_filepath: str or None
          Path of the converted pickle file. Default: `None`
        col_str: list of str
          Name of the variables to extract from the LiPD files

    Returns:
    -------
        df: `pandas.DataFrame`
          Converted Pandas DataFrame
    """

    # Save the current working directory for later use, as the LiPD utility will change it in the background
    work_dir = os.getcwd()
    # LiPD utility requries the absolute path
    lipd_dirpath = os.path.abspath(lipd_dirpath)
    # Load LiPD files
    lipds = lipd.readLipd(lipd_dirpath)
    # Extract timeseries from the list of LiDP objects
    ts_list = lipd.extractTs(lipds)
    # Recover the working directory
    os.chdir(work_dir)
    # Create an empty pandas.DataFrame with the number of rows to be the number of the timeseries (PAGES2k records),
    # and the columns to be the variables we'd like to extract
    df_tmp = pd.DataFrame(index=range(len(ts_list)), columns=col_str)
    # Loop over the timeseries and pick those for global temperature analysis
    i = 0
    for ts in ts_list:
        if (
            "paleoData_useInGlobalTemperatureAnalysis" in ts.keys()
            and ts["paleoData_useInGlobalTemperatureAnalysis"] == "TRUE"
        ):
            for name in col_str:
                try:
                    df_tmp.loc[i, name] = ts[name]
                except:
                    df_tmp.loc[i, name] = np.nan
            i += 1
    # Drop the rows with all NaNs (those not for global temperature analysis)
    df = df_tmp.dropna(how="all")
    # Save the dataframe to a pickle file for later use
    if pkl_filepath:
        save_path = os.path.abspath(pkl_filepath)
        print(f"Saving pickle file at: {save_path}")
        df.to_pickle(save_path)
    return df

##  Helper functions


In [ ]:
class SupressOutputs(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self

    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio  # free up some memory
        sys.stdout = self._stdout##  Helper functions


# Spectral Analysis of Paleoclimate Data

## Spectral Analysis of Paleoclimate Data

In this section, we will manipulate paleoclimate proxy datasets using previously learned computational tools, and apply spectral analysis to further interpret the data.

We should:

*   Interpret the [LR04](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2004PA001071) benthic δ<sup>18</sup>O curve and how it records temperature
*   Perform various spectral analysis methods to assess dominant spectral powers in the LR04 data
*   Interpret variations in glacial-interglacial cycles recorded by the LR04 δ<sup>18</sup>O record

## Plotting the LR04 δ<sup>18</sup>O benthic stack

We will be analyzing a δ<sup>18</sup>O data from LR04: [Lisiecki, L. E., and Raymo, M. E. (2005)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2004PA001071).

This is a stack of 57 globally distributed records of δ<sup>18</sup>O from benthic foraminifera that spans the past ~5 million years.

δ<sup>18</sup>O of foraminifera records temperature due to differences in the isotopic composition of the ocean during glacial and interglacial periods. 

The δ<sup>18</sup>O of the ocean (and forams) is more depleted (takes on a smaller values) during interglacials and more enriched (takes on a larger value) during glacials.

## Plotting the LR04 δ<sup>18</sup>O benthic stack

Let's start by importing the data:

In [ ]:
# Donwload the data
filename_LR04 = "LR04.csv"
url_LR04 = (
    "https://raw.githubusercontent.com/LinkedEarth/PyleoTutorials/main/data/LR04.csv"
)
lr04_data = pd.read_csv(
    pooch_load(filelocation=url_LR04, filename=filename_LR04), skiprows=4
)
lr04_data.head()

## Plotting the LR04 δ<sup>18</sup>O benthic stack

We can now create a `pyleo Series` object containing the data:

In [ ]:
ts_lr04 = pyleo.Series(
    time=lr04_data["Time (ka)"],
    value=lr04_data["Benthic d18O (per mil)  "],
    time_name="Age",
    time_unit="kyr BP",
    value_name="Benthic d18O (per mil)",
    value_unit="\u2030",
    label="LR04",
)

## Plotting the LR04 δ<sup>18</sup>O benthic stack

This record spans the past ~5 million years (5,000 kyr), but we're going to focus in on the past 1 million years (1,000 kyr), so let's create a time slice of just this time period, and plot the time series.

In [ ]:
lr04_slice = ts_lr04.slice([0, 1000])
fig, ax = plt.subplots()  # assign a new plot axis
lr04_slice.plot(ax=ax, legend=False, invert_yaxis=True)

## Your turn

1. What patterns do you observe in the record?
2. Does the amplitude of the glacial-interglacial cycles vary over time?
3. At what frequency do these glacial-interglacial cycles occur?

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

To better assess the dominant temporal patterns in this record, you can use spectral analysis.

Spectral analysis is a useful data analysis tool in paleoclimate because it allows us to discover underlying periodicities in time series data and can help establish quantitative relationships between forcings and climate variations.

Let's explore various spectral analysis methods and apply them to the LR04 record to interpret changes in the frequency of glacial-interglacial cycles.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Spectral Density

Pyleoclim enables five methods to estimate the [spectral density](https://glossary.ametsoc.org/wiki/Spectral_density), which will be our main interest in this tutorial:

*   **Basic Periodogram**, which uses a Fourier transform. The method has various windowing available to reduce variance.
*  **Welch's periodogram**, a variant of the basic periodogram, which uses Welch's method of overlapping segments. The periodogram is computed on each segment and averaged together.
*   **Multi-taper method (MTM)**, which attempts to reduce the variance of spectral estimates by using a small set of tapers rather than the unique data taper or spectral window.
*   **Lomb-Scargle periodogram**, an inverse approach designed for unevenly-spaced datasets. Several windows are available and Welch's segmentation can also be used with this method.
*   **Weighted wavelet Z-transform (WWZ)**, a wavelet-based method also made for unevenly-spaced datasets.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Spectral Density

All of these methods are available through [`Series.spectral()`](https://pyleoclim-util.readthedocs.io/en/latest/core/api.html#pyleoclim.core.series.Series.spectral) by changing the `method` argument.

Here we will focus on Lomb-Scargle and Weighted wavelet Z-transform in more detail. For additional information on the other methods, refer to [this notebook from Pyleoclim](https://github.com/LinkedEarth/PyleoTutorials/blob/main/notebooks/L2_spectral_analysis.ipynb).

To get a sense of how the estimates of the spectral density using each of these methods compare, we can plot them on the same graph below.

Note we must first interpolate to an even grid and then standardize before estimating the spectral density.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Spectral Density

In [ ]:
fig, ax = plt.subplots()

# basic periodogram
lr04_slice.interp(step=0.5).standardize().spectral(method="periodogram").plot(
    ax=ax, xlim=[1000, 5], ylim=[0.001, 1000], label="periodogram"
)
# Welch's periodogram
lr04_slice.interp(step=0.5).standardize().spectral(method="welch").plot(
    ax=ax, xlim=[1000, 5], ylim=[0.001, 1000], label="welch"
)
# Multi-taper Method
lr04_slice.interp(step=0.5).standardize().spectral(method="mtm").plot(
    ax=ax, xlim=[1000, 5], ylim=[0.001, 1000], label="mtm"
)
# Lomb-Scargle periodogram
lr04_slice.interp(step=0.5).standardize().spectral(method="lomb_scargle").plot(
    ax=ax, xlim=[1000, 5], ylim=[0.001, 1000], label="lomb_scargle"
)
# weighted wavelet Z-transform (WWZ)
lr04_slice.interp(step=0.5).standardize().spectral(method="wwz").plot(
    ax=ax, xlim=[1000, 5], ylim=[0.001, 1000], label="wwz"
)

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Spectral Density

Which method should you choose?

The Lomb-Scargle periodogram is the default method in Pyleoclim, and is a good choice for many applications because: 

1. It works with unevenly-spaced timeseries which are often encountered in paleoclimate data
2. It seems to give consistent results over parameter ranges
3. It does not require interpolation, limiting timeseries manipulation
4. It is fairly fast to compute.

We will choose this estimate to analyze.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Spectral Density

In [ ]:
# Lomb-Scargle periodogram
lr04_ls_sd = lr04_slice.interp(step=0.5).standardize().spectral(method="lomb_scargle")

lr04_ls_sd.plot(xlim=[1000, 5], ylim=[0.001, 1000], label="lomb_scargle")

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Significance Testing

Note that there are few peaks that seem more dominant than others, but which of these peaks are significant?

> That is, do they stand out more than peaks would from a random time series? 

To figure that out, we can use the method `signif_test`.

Pyleoclim generates many *surrogates* of the time series using an AR(1) (Autoregressive Process of order 1 - i.e. a random process). 

These surrogates serve to identify the probability of a peak not just being from random noise and the likelihood that the peak of the same magnitude would be unlikely to be present in a random dataset. 

The default number of surrogates generated is 200, and the larger this value, the more smooth and precise our results are.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Significance Testing

In [ ]:
lr04_ls_sd_sig = lr04_ls_sd.signif_test()

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Significance Testing

In [ ]:
lr04_ls_sd_sig.plot(xlabel="Period [kyrs]")

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Significance Testing

The variable `lr04_ls_sd_sig` [PSD object](https://pyleoclim-util.readthedocs.io/en/latest/core/api.html#pyleoclim.core.psds.PSD) contains the significant frequencies of the significant peaks from the spectral density of the LR04 data. 

We can now create the same plot of spectral power, but with only the significant peaks. 

**Note**: For this figure, we will plot the x-axis a bit differently. 

In the previous plot, the x-axis was the "period" but this time the x-axis will be "frequency", which is the inverse of the period.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Significance Testing

In [ ]:
fig, ax = plt.subplots()
ax.plot(lr04_ls_sd_sig.frequency, lr04_ls_sd_sig.amplitude)
ax.set_xlim([0.005, 0.07])
ax.set_ylim([0, 200])
ax.set_xlabel("Frequency [1/kyr]")
ax.set_ylabel("Spectra Amplitude")

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Significance Testing

The largest spectral powers in the data occur at a frequencies of ~0.01 and ~0.025, as well as a smaller peak at ~0.042, which correspond to 100 kyr, 40 kyr, and 23 kyr, respectively. 

> Do those periodicities sound familiar? 

These are the cycles of **eccentricity, obliquity and precession**. 

The presence of spectral powers in the LR04 data at the eccentricity, obliquity and precession frequencies highlights the influence of orbital forcings on glacial-interglacial cycles.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Wavelet Analysis

Another related tool we can use to learn more about the climate variability recorded in the data is wavelet analysis. 

This method allows us to  "unfold" a spectrum and look at its evolution over time. 

> In other words, wavelet analysis can help us determine changes in the spectral power over time. 

For additional details about wavelet analysis, refer to [this guide](https://paos.colorado.edu/research/wavelets/bams_79_01_0061.pdf).

There are several ways to access this functionality in Pyleoclim, but here we use `summary_plot`.

`summary_plot` stacks together the timeseries itself in the upper panel, its scalogram (a plot of the magnitude of the wavelet power) in the middle, and the power spectral density (PSD, which we plotted earlier in this tutorial) obtained from summing the wavelet coefficients over time to the right.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Wavelet Analysis

In [ ]:
# create a scalogram
scal = lr04_slice.interp(step=0.5).standardize().wavelet()

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Wavelet Analysis

In [ ]:
# make plot
lr04_slice.summary_plot(
    psd=lr04_ls_sd_sig, scalogram=scal, time_lim=[0, 1000], psd_label="Amplitude"
)

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Wavelet Analysis

In this wavelet spectrum, the age of the record is on the x-axis, the period is on the y-axis, and the color represents the amplitude of that power spectrum, with yellow indicating a higher power and purple indicating a lower power. 

The time series on top is the original LR04 δ<sup>18</sup>O  data, and the plot on the right is the spectral analysis and significance test figure we created earlier in this tutorial.

## Spectral analysis of the LRO4 δ<sup>18</sup>O benthic stack

### Wavelet Analysis (your turn)

In the spectral analysis above, the 100 kyr and 40 kyr period are the most dominant. 

Here, we further see that over the past 1 million years, the 100 kyr cycle is the strongest (as seen by the yellow color at the 100 kyr scale), followed by the 40 kyr cycle (as seen by the light purple color at the 40 kyr scale). 

You may notice an absence of much color at the 23 kyr scale. 

***What does this suggest about the influence of precession on glacial-interglacial cycles on this timescale?***

# Assessing Climate Forcings

## Assessing Climate Forcings

Climate forcings are essential to assess the variations observed in paleoclimate records. 

Analyze climate forcings include:

*   Plot and interpret temperature reconstructions from speleothem oxygen isotopes
*   Generate and plot time series of solar insolation
*   Assess the orbital forcings on monsoon intensity over the past 400,000 years using spectral analysis

## Understanding and Interpreting Climate Forcings

A common task in paleoclimatology is to relate a proxy record (or several of them) to the particular forcing(s) that is thought to dominate that particular record (e.g., based on the proxy, location, etc.)

We've already spent some time learning about the influence of Earth's orbital configuration on glacial-interglacial cycles. 

Now we will assess the the climate forcings of monsoon intensity over the past 400,000 years. 

Recall that monsoons are seasonal changes in the direction of strongest wind and precipitation that are primarily driven by variations in seasonal insolation.

## Understanding and Interpreting Climate Forcings

Land and the ocean have different abilities to hold onto heat. Land cools and warms much faster than the ocean does due to high heat capacity. 

This temperature difference leads to a pressure difference that drives atmospheric circulations called monsoons. 

*   **Summer (Northern Hemisphere)**: land is warmer than the ocean, so the winds blow towards the land, resulting in heavy rainfall over land.
*   **Winter (Northern Hemisphere)**: land is cooler than the ocean, so the winds blow away from the land, resulting in heavy rainfall over the ocean and decreased rainfall over land.
 
On longer timescales, changes in insolation and the mean climate state can drive changes in monsoon intensity.

## Understanding and Interpreting Climate Forcings

To assess these long-term changes, we can analyze paleoclimate reconstructions from monsoon regions such as India, Southeast Asia or Africa.

δ<sup>18</sup>O records from speleothems in Chinese caves are broadly interpreted to reflect continental-scale monsoon circulations. 

We will plot and analyze a composite of three δ<sup>18</sup>O speleothem records from multiple caves in China (Sanbao, Hulu, and Dongge caves) from [Cheng et al. (2016)](https://hwww.nature.com/articles/nature18591).

We will then assess the relationship between the climate signals recorded by the speleothem δ<sup>18</sup>O and solar insolation.

## Understanding and Interpreting Climate Forcings

First, we download and plot the speleothem oxygen isotope data:

In [ ]:
# download the data from the url
filename_Sanbao_composite = "Sanbao_composite.csv"
url_Sanbao_composite = "https://raw.githubusercontent.com/LinkedEarth/paleoHackathon/main/data/Orbital_records/Sanbao_composite.csv"
data = pd.read_csv(
    pooch_load(filelocation=url_Sanbao_composite, filename=filename_Sanbao_composite)
)

# create a pyleo.Series
d18O_data = pyleo.Series(
    time=data["age"] / 1000,
    time_name="Age",
    time_unit="kyr BP",
    value=-data["d18O"],
    value_name=r"$\delta^{18}$O",
    value_unit="\u2030",
)
d18O_data.plot()

## Understanding and Interpreting Climate Forcings

You may notice that in the figure we just made, the δ<sup>18</sup>O values on the y-axis is plotted with more positive values up.

In the previous plots, we have plotted isotopic data with more negative values up (since more negative/"depleted" suggests warmer temperatures or increased rainfall).

However, the pre-processed δ<sup>18</sup>O data that we're using here was multipled by $-1$, so now a more positive/"enriched" value suggests warmer temperatures or increased rainfall.

In other words, in this figure, **upward on the y-axis is *increased* monsoon intensity** and **downward on the y-axis is *decreased* monsoon intensity**.

## Understanding and Interpreting Climate Forcings

Let's perform spectral analysis on the speleothem oxygen isotope data.

Recall that spectral analysis can help us identify dominant cyclicities in the data, which can be useful for assessing potential climate forcings.

Here we'll use the Weighted Wavelet Z-Transform (WWZ) method you learned in the previous slides:

## Understanding and Interpreting Climate Forcings

Standardize the data:

In [ ]:
d18O_stnd = d18O_data.interp(step=0.5).standardize()  # save it for future use

## Understanding and Interpreting Climate Forcings

Calculate the WWZ spectral analysis:

In [ ]:
d18O_wwz = d18O_stnd.spectral(method="wwz")

## Understanding and Interpreting Climate Forcings

Plot WWZ results:

In [ ]:
d18O_wwz.plot(xlim=[100, 5], ylim=[0.001, 1000])

## Understanding and Interpreting Climate Forcings: Your turn

The dominant spectral power is at ~23,000 years.

This suggests a link between monsoon intensity and orbital precession.

**Is this peak significant?**

Use the skills you learned in the previous slides to test the significance of this peak at the 95% confidence level.

For this exercise, input `number = 30` as the default value which will take a long time to run.

In [ ]:
# perform significance test with 5 surrogates
d18O_wwz_sig = ...

# plot the results
_ = ...

## Constructing Insolation Curves

To further explore and confirm the relationship between monsoon intensity and orbital precession, let's take a look at insolation data and compare this to the speleothem δ<sup>18</sup>O records from Asia.

Insolation refers to the amount of solar radiation reaching a given area of the Earth’s surface over a specific period.

Recall that insolation is controlled by variations in Earth's orbital cycles (eccentricity, obliquity, precession), so by comparing the δ<sup>18</sup>O record to insolation, we can assess the influence of orbital variations on δ<sup>18</sup>O and monsoon intensity. 

To compute solar insolation, we can use the package [`climlab`](https://climlab.readthedocs.io/en/latest/index.html) by Brian Rose.

Let's create a time series over the past 400,000 years of changes in summer insolation at 31.67ºN, which is the latitude of Sanbao, one of the caves from which the speleothem records were produced.

## Constructing Insolation Curves

In [ ]:
# specify time interval and units
kyears = np.linspace(-400, 0, 1001)

# subset of orbital parameters for specified time
orb = OrbitalTable.interp(kyear=kyears)
days = np.linspace(0, const.days_per_year, 365)

# generate insolation at Sanbao latitude (31.67)
Qsb = daily_insolation(31.67, days, orb)

# julian days 152-243 are JJA
Qsb_jja = np.mean(Qsb[:, 151:243], axis=1)

## Constructing Insolation Curves

Now we can store this data as a `Series` in Pyleoclim and plot the data versus time:

In [ ]:
ts_qsb = pyleo.Series(
    time=-kyears,
    time_name="Age",
    time_unit="ky BP",
    value=Qsb_jja,
    value_name="JJA Insolation",
    value_unit=r"$W.m^{-2}$",
)

ts_qsb.plot()

## Constructing Insolation Curves

Next, let's plot and compare the speleothem δ<sup>18</sup>O data and the solar insolation data:

In [ ]:
# standardize the insolation data
ts_qsb_stnd = ts_qsb.standardize()

# create a MultipleSeries of the speleothem d18O record and insolation data
compare = [d18O_stnd, ts_qsb_stnd]
ms_compare = pyleo.MultipleSeries(compare, time_unit="kyr BP", name=None)

# create a stackplot to compare the data
ms_compare.stackplot()

## Constructing Insolation Curves

By visually comparing the time series of the two records, we can see similarites at orbital scales. 

To confirm this, we can use spectral analysis to determine the dominant spectral power of the insolation data:

Calculate the WWZ spectral analysis:

In [ ]:
psd_wwz = ts_qsb_stnd.spectral(method="wwz")

## Constructing Insolation Curves

In [ ]:
psd_wwz.plot()

## Constructing Insolation Curves: Your turn

1. What is the dominant spectral power in summer insolation at 31ºN latitude? How does this compare to the speleothem data?

2. Why might there be a relationship between solar insolation and monsoon intensity? What does the common spectral power in both the insolation and δ<sup>18</sup>O records suggest about the climate forcings driving monsoon intensity in this region?

## Questions?

## See you in the next lecture!